In [133]:
print("Hello World!")

Hello World!


In [134]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [135]:
import pandas as pd

# Try reading the CSV file with a different encoding
try:
    store = pd.read_csv("Sample-Superstore.csv", encoding='latin-1')
except UnicodeDecodeError:
    print("Error: Unable to read the file with 'latin-1' encoding.")
    try:
        store = pd.read_csv("Sample-Superstore.csv", encoding='utf-16')
    except UnicodeDecodeError:
        print("Error: Unable to read the file with 'utf-16' encoding.")

In [136]:
store.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


In [137]:
store.rename(columns={"Order ID": "OrderID", "Order Date": "OrderDate", "Ship Date": "ShipDate", "Ship Mode": "ShipMode", "Customer ID": "CustomerID", "Customer Name": "CustomerName", "Postal Code": "PostalCode", "Product ID": "ProductID","Sub-Category":"SubCategory", "Product Name": "ProductName"}, inplace=True)


In [138]:
store_copied= store.copy()

## Creating  new columns

In [139]:


store_copied['AdjustedTotalSales'] = ((store_copied['Sales'] / (1 - (store_copied['Discount']))) + store_copied['Profit']).round(2)


store_copied['ProductPrice'] = (store_copied['AdjustedTotalSales'] / store_copied['Quantity']).round(2)



store_copied['SupplierPrice'] = (((store_copied['Sales'] - store_copied['Profit'] + store_copied['Discount']) / (store_copied['Quantity'] * (1 - store_copied['Discount'])))).round(2)






#Basic Calc.
##SupplierPrice= ProductPrice-Profit
##ProductPrice=Sales/Qty ### Of course when there no Discount




# Calculate Supplier Price
##store_copied['SupplierPrice'] = ((store_copied['Sales'] + store_copied['Discount']) / store_copied['Quantity']).round(2)


In [6]:
# Creating new columns Unitprice and Costprice while rounding it to 2 decimals


#1#store_copied['ProductPrice'] = ((store_copied['Sales'] / (1 - store_copied['Discount'])) / store_copied['Quantity']).round(2)


#2#store_copied['SupplierPrice'] = ((store_copied['Sales'] / store_copied['Quantity']) - store_copied['Profit'] + store_copied['Discount']).round(2)




#store_copied['UnitPrice'] = ((store_copied['Sales'] - store_copied['Discount']) / store_copied['Quantity']).round(2)

#store_copied['CostPrice']= ((store_copied['Sales'] - store_copied['Profit']) - store_copied['Discount']).round(2)
## store_copied['CostPrice']= ((store_copied['Sales'] - store_copied['Profit']) - store_copied['Discount']).round(2)

## divide by qty


##SupplierPrice= ProductPrice-Profit
##ProductPrice=Sales/Qty ### Of course when there no Discount



In [38]:
store_copied.tail(5)

,Row ID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,AdjustedTotalSales,ProductPrice,SupplierPrice
9989,9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,Furniture,Furnishings,Ultra Door Pull Handle,25.248,3,0.2,4.1028,35.66,11.89,8.89
9990,9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.960,2,0.0,15.6332,107.59,53.80,38.16
9991,9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Technology,Phones,Aastra 57i VoIP phone,258.576,2,0.2,19.3932,342.61,171.30,149.61
9992,9993,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.600,4,0.0,13.3200,42.92,10.73,4.07
9993,9994,CA-2017-119914,5/4/2017,5/9/2017,Second Class,CC-12220,Chris Cortes,Consumer,United States,Westminster,...,Office Supplies,Appliances,"Acco 7-Outlet Masterpiece Power Center, Wihtou...",243.160,2,0.0,72.9480,316.11,158.06,85.11


## Creation of Tables 

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('VS GROUP.db')
cursor = conn.cursor()

# Create customers table with unique CustomerID
cursor.execute('''CREATE TABLE customers (
                    CustomerID VARCHAR PRIMARY KEY,
                    CustomerName VARCHAR,
                    Segment TEXT,
                    UNIQUE(CustomerID)  -- Ensure CustomerID is unique
                )''')

# Create address table with foreign key reference to customers and unique AddressID
cursor.execute('''CREATE TABLE address (
                    AddressID INTEGER PRIMARY KEY,
                    CustomerID VARCHAR,
                    Country VARCHR,
                    Region VARCHAR,
                    State VARCHAR,
                    City VARCHAR,
                    PostalCode INTEGER,
                    FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID),
                    UNIQUE(AddressID)  -- Ensure AddressID is unique
                )''')

# Create products table with unique ProductID
cursor.execute('''CREATE TABLE products (
                    ProductID VARCHAR PRIMARY KEY,
                    Category TEXT,
                    SubCategory TEXT,
                    ProductName VARCHAR,
                    ProductPrice REAL,
                    Discount REAL,
                    SupplierPrice REAL,
                    UNIQUE(ProductID)  -- Ensure ProductID is unique
                )''')

# Create orders table with primary key, foreign key reference to customers, and unique OrderID
cursor.execute('''CREATE TABLE orders (
                    OrderID VARCHAR PRIMARY KEY,
                    OrderDate DATE,
                    CustomerID VARCHAR,
                    FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID),
                    UNIQUE(OrderID)  -- Ensure OrderID is unique
                )''')

# Create order_details table with primary key, foreign key reference to orders and products
cursor.execute('''CREATE TABLE order_details (
                    OrderDetailsID INTEGER PRIMARY KEY,
                    OrderID VARCHAR,
                    ProductID  VARCHAR, 
                    Quantity INTEGER,
                    FOREIGN KEY (OrderID) REFERENCES orders(OrderID),
                    FOREIGN KEY (ProductID) REFERENCES products(ProductID)
                )''')

# Create shipments table with primary key, foreign key reference to customers and products
cursor.execute('''CREATE TABLE shipments (
                    ShipID INTEGER PRIMARY KEY,
                    ShipDate TEXT,
                    ShipMode TEXT,
                    CustomerID VARCHAR, 
                    ProductID VARCHAR, 
                    FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID),
                    FOREIGN KEY (ProductID) REFERENCES products(ProductID)
                )''')

# Commit changes and close connection
conn.commit()
conn.close()


## Splitting of Data to Table

## Orders

In [143]:
ord=store_copied[["OrderID","OrderDate","CustomerID"]] 

In [144]:
## dropping duplicates

ord.drop_duplicates(inplace=True)
order_f=ord.reset_index(drop=True)

C:\Users\godwi\AppData\Local\Temp\ipykernel_6920\2920754027.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ord.drop_duplicates(inplace=True)


In [145]:

orders= order_f[["OrderID","OrderDate","CustomerID"]] 
# Connect to the SQLite database
conn = sqlite3.connect('VS GROUP.db')

# Write DataFrames to SQLite database
orders.to_sql('orders', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()




## Customers

In [146]:
cus=store_copied[["CustomerID", "CustomerName","Segment"]]

In [147]:
## dropping duplicates

cus.drop_duplicates(inplace=True)
customer_f=cus.reset_index(drop=True)

C:\Users\godwi\AppData\Local\Temp\ipykernel_6920\1124437389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cus.drop_duplicates(inplace=True)


In [148]:

customers=customer_f[["CustomerID","CustomerName","Segment"]] 
# Connect to the SQLite database
conn = sqlite3.connect('VS GROUP.db')

# Write DataFrames to SQLite database
customers.to_sql('customers', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()




## Address

In [163]:
store_copied[["CustomerID","Country","Region","State","City","PostalCode"]].nunique()

CustomerID    793
Country         1
Region          4
State          49
City          531
PostalCode    631
dtype: int64

In [149]:
address_df=store_copied[["CustomerID","Country","Region","State","City","PostalCode"]]

In [150]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect('VS GROUP.db')


# Extract specific columns into a new DataFrame
address=address_df[["CustomerID","Country","Region","State","City","PostalCode"]]# Write the DataFrame to the SQLite database
address.to_sql('address', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()

In [ ]:
import sqlite3

# SQL script to add a new column named "addressID" and populate it with numbers starting from 1
sql_script = """
-- Add a new column named "addressID"
ALTER TABLE address ADD COLUMN AddressID INTEGER;

-- Populate the addressID column with numbers starting from 1
UPDATE address
SET AddressID = ROWID;
"""

# Connect to SQLite database
conn = sqlite3.connect('VS GROUP.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL script
cursor.executescript(sql_script)

# Commit the changes to the database
conn.commit()

# Close cursor and connection
cursor.close()
conn.close()


## Products

In [152]:
store_product=store_copied.copy()

In [153]:
# Drop duplicate ProductIDs
store_copied_no_duplicates = store_product.drop_duplicates(subset=['ProductID'])

# Create a new column for UniqueProductID based on the combination of ProductID and ProductName
store_copied_no_duplicates['UniqueProductID'] = store_copied_no_duplicates['ProductID'].astype(str) + '_' + store_copied_no_duplicates['ProductName']

store_copied_no_dup= store_copied_no_duplicates.reset_index(drop=True)


C:\Users\godwi\AppData\Local\Temp\ipykernel_6920\2614288099.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_copied_no_duplicates['UniqueProductID'] = store_copied_no_duplicates['ProductID'].astype(str) + '_' + store_copied_no_duplicates['ProductName']


In [112]:
store_copied_no_dup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1862 entries, 0 to 1861
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Row ID              1862 non-null   int64  
 1   OrderID             1862 non-null   object 
 2   OrderDate           1862 non-null   object 
 3   ShipDate            1862 non-null   object 
 4   ShipMode            1862 non-null   object 
 5   CustomerID          1862 non-null   object 
 6   CustomerName        1862 non-null   object 
 7   Segment             1862 non-null   object 
 8   Country             1862 non-null   object 
 9   City                1862 non-null   object 
 10  State               1862 non-null   object 
 11  PostalCode          1862 non-null   int64  
 12  Region              1862 non-null   object 
 13  ProductID           1862 non-null   object 
 14  Category            1862 non-null   object 
 15  SubCategory         1862 non-null   object 
 16  Produc

In [154]:
store_copied_no_dup[["ProductID","ProductName", "Category","SubCategory","ProductPrice","Discount", "SupplierPrice"]]

,ProductID,ProductName,Category,SubCategory,ProductPrice,Discount,SupplierPrice
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases,151.94,0.00,110.02
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs,317.17,0.00,170.79
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels,10.74,0.00,3.87
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables,271.60,0.45,487.66
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies,Storage,15.24,0.20,12.53
...,...,...,...,...,...,...,...
1857,TEC-PH-10002817,RCA ViSYS 25425RE1 Corded phone,Technology,Phones,147.14,0.20,119.89
1858,TEC-MA-10003589,Cisco 8961 IP Phone Charcoal,Technology,Machines,194.95,0.70,433.99
1859,OFF-AP-10003099,"Eureka Hand Vacuum, Bagless",Office Supplies,Appliances,63.27,0.00,35.59
1860,TEC-PH-10002645,LG G2,Technology,Phones,424.99,0.40,625.12


In [125]:
store_copied_no_dup.head(2)

,Row ID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,SubCategory,ProductName,Sales,Quantity,Discount,Profit,AdjustedTotalSales,ProductPrice,SupplierPrice,UniqueProductID
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136,303.87,151.94,110.02,FUR-BO-10001798_Bush Somerset Collection Bookcase
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820,951.52,317.17,170.79,FUR-CH-10000454_Hon Deluxe Fabric Upholstered ...


In [155]:
products_f=store_copied_no_dup[["ProductID","ProductName","Category","SubCategory","ProductPrice","Discount","SupplierPrice"]]

In [131]:
products_f.head()

,ProductID,ProductName,Category,SubCategory,ProductPrice,Discount,SupplierPrice
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases,151.94,0.00,110.02
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs,317.17,0.00,170.79
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels,10.74,0.00,3.87
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables,271.60,0.45,487.66
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies,Storage,15.24,0.20,12.53


In [156]:
# Connect to SQLite database
conn = sqlite3.connect('VS GROUP.db')

# Extract specific columns into a new DataFrame
products=products_f[["ProductID","ProductName","Category","SubCategory","ProductPrice","Discount","SupplierPrice"]]# Write the DataFrame to the SQLite database
products.to_sql('products', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()


##  Order_Details

In [157]:
order_details_df=store_copied[["OrderID","ProductID","Quantity"]]

In [158]:
# Connect to SQLite database
conn = sqlite3.connect('VS GROUP.db')


# Extract specific columns into a new DataFrame
order_details=order_details_df[["OrderID","ProductID","Quantity"]]
order_details.to_sql('order_details', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()


In [ ]:
import sqlite3

# SQL script to add a new column named "orderdetailsID" and populate it with numbers starting from 1
sql_script = """
-- Add a new column named "orderdetailsID"
ALTER TABLE order_details ADD COLUMN OrderDetailsID INTEGER;

-- Populate the orderdetailsID column with numbers starting from 1
UPDATE order_details
SET OrderDetailsID = ROWID;
"""

# Connect to SQLite database
conn = sqlite3.connect('VS GROUP.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL script
cursor.executescript(sql_script)

# Commit the changes to the database
conn.commit()

# Close cursor and connection
cursor.close()
conn.close()


## Shipments

In [160]:
shipments_df=store_copied[["ShipDate","ShipMode", "CustomerID", "ProductID"]]

In [161]:
# Connect to SQLite database
conn = sqlite3.connect('VS GROUP.db')


# Extract specific columns into a new DataFrame
shipments=shipments_df[["ShipDate","ShipMode", "CustomerID", "ProductID"]]
shipments.to_sql('shipments', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()


In [ ]:
import sqlite3

# SQL script to add a new column named "ShipID" and populate it with numbers starting from 1
sql_script = """
-- Add a new column named "ShipID"
ALTER TABLE shipments ADD COLUMN ShipID INTEGER;

-- Populate the ShipID column with numbers starting from 1
UPDATE shipments
SET ShipID = ROWID;
"""

# Connect to SQLite database
conn = sqlite3.connect('VS GROUP.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL script
cursor.executescript(sql_script)

# Commit the changes to the database
conn.commit()

# Close cursor and connection
cursor.close()
conn.close()


In [ ]:
conn = sqlite3.connect('Superstore.db')

In [ ]:
cursor = conn.cursor()

In [ ]:
# Perform a query

cursor.execute("SELECT * FROM Customers WHERE Segment = 'Corporate'")
rows = cursor.fetchall()

In [ ]:
# Convert the retrieved data into a Dataframe
Corporate_customers = pd.DataFrame([row[0:2] for row in rows], columns= ['CustomerID', 'CustomerName'])

In [ ]:
Corporate_customers

,CustomerID,CustomerName
0,DV-13045,Darrin Van Huff
1,KB-16585,Ken Black
2,KB-16585,Ken Black
3,GH-14485,Gene Hale
4,GH-14485,Gene Hale
...,...,...
3015,HE-14800,Harold Engle
3016,HE-14800,Harold Engle
3017,HE-14800,Harold Engle
3018,RA-19885,Ruben Ausman


In [ ]:
# Close the connection
conn.close()

In [ ]:
import sqlite3

# SQL script to add a new column named "addressID" and populate it with numbers starting from 1
sql_script = """
-- Add a new column named "addressID"
ALTER TABLE your_table ADD COLUMN addressID INTEGER;

-- Populate the addressID column with numbers starting from 1
UPDATE your_table
SET addressID = ROWID;
"""

# Connect to SQLite database
conn = sqlite3.connect('your_database.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL script
cursor.executescript(sql_script)

# Commit the changes to the database
conn.commit()

# Close cursor and connection
cursor.close()
conn.close()


In [ ]:
order_details.head(2)

,Order ID,Product ID,Sales,Quantity
0,CA-2016-152156,FUR-BO-10001798,261.96,2
1,CA-2016-152156,FUR-CH-10000454,731.94,3


In [ ]:
products.head(2)

,Product ID,Category,Sub-Category,Product Name,Sales,Discount,Profit
0,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,0.0,41.9136
1,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,0.0,219.5820


In [ ]:
import mysql.connector


ModuleNotFoundError: No module named 'mysql'

In [ ]:
import pandas as pd
import mysql.connector

# Connect to MySQL database
conn = mysql.connect(
    host="Oti Godwin",
    user="otiG30",
    password="12345",
    database="mysql_E_Commerce"
)

# Assuming store_copied is your DataFrame
# Extract specific columns into a new DataFrame
orders = store_copied[["Order ID", "Order Date", "Ship Date", "Ship Mode", "Customer ID"]]

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create a table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS orders (
        OrderID INT,
        OrderDate DATE,
        ShipDate DATE,
        ShipMode VARCHAR(255),
        CustomerID INT
    )
""")

# Insert DataFrame into MySQL table
for index, row in orders.iterrows():
    cursor.execute("""
        INSERT INTO orders (OrderID, OrderDate, ShipDate, ShipMode, CustomerID)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['Order ID'], row['Order Date'], row['Ship Date'], row['Ship Mode'], row['Customer ID']))

# Commit changes
conn.commit()

# Close cursor and connection
cursor.close()
conn.close()


ModuleNotFoundError: No module named 'mysql'